In [ ]:
pip install vk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import vk
from vk.session import requests
import json

In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [ ]:
import copy
import time 

In [ ]:
import networkx as nx
import matplotlib as plt
plt.rcParams['figure.figsize'] = [50, 50]

In [ ]:
from google.colab import files
FileRead = open('/content/token.txt', 'r')
token = FileRead.read()#Вывод первых 100 символов
# в конце файл необходимо закрыть
FileRead.close()
version = 5.126
users_idGroup = [181948702, 154308430, 152087036, 143779216, 185215592, 161594007, 472583599, 113001678, 183648122, 196404046, 
    85770676, 355133495, 137872152, 102711908, 126568936, 507184348, 458357505, 131765293, 323208790, 173792332, 343410069, 
    346034855, 255053910, 338686058, 312616596]

In [ ]:
def RequestsUsersGet(users_id, i):
  response = requests.get("https://api.vk.com/method/users.get",
                        params={
                            "user_id": users_id[i],
                            "name_case": "nom",
                            "fields": "nickname",
                            "access_token": token,
                            "v": 5.131
                        }
                        )  
  if(response.json()):
    data = response.json()
  return data

In [ ]:
def RequestsFriendsGet(users_id, j):
  if(j %  100 == 0):
    time.sleep(3)
  response = requests.get("https://api.vk.com/method/friends.get",
                        params={
                            "user_id": users_id[j],
                            "order": "name",
                            "count": 5000,
                            "offset": 0,
                            "fields": "nickname",

                            "access_token": token,
                            "v": version
                        }
                        )
  if(response.json()): #написать проверку здесь
    data = response.json()
  return data

In [ ]:
frameGroup = pd.DataFrame(columns=['main_id', 'first_name', 'last_name'])
frameFriends = pd.DataFrame(columns=['main_id', 'id_friend', 'first_name', 'last_name'])
frameFriendsOfFriends = pd.DataFrame(columns=['main_id', 'id_friend', 'first_name', 'last_name'])
GraphCentrality = nx.Graph()

#1 уровень (получаем информацию о одногруппниках)
for i in range(len(users_idGroup)):
  try: dataGroup = RequestsUsersGet(users_idGroup, i)
  except: continue
  try: 
    if dataGroup['response'][0]['is_closed'] == True: 
      ClosedAcc = "Доступ к странице пользователя недоступен"
    else: 
      frameGroup = frameGroup.append({'main_id': dataGroup['response'][0]['id'], 'first_name': dataGroup['response'][0]['first_name'], 'last_name': dataGroup['response'][0]['last_name']}, ignore_index=True)
  except:
    errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"

#2 уровень (получаем информациою о друзьях одногруппников)
for j in range(len(frameGroup.main_id)):
  try: dataFriends = RequestsFriendsGet(frameGroup.main_id, j)
  except: continue
  countOfUserFriends = dataFriends['response']['count']
  for jx in range(countOfUserFriends):
    try:
      if dataFriends['response']['items'][jx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      if dataFriends['response']['items'][jx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    except:  
      try: 
        if dataFriends['response']['items'][jx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        else: 
          frameFriends = frameFriends.append({'main_id': frameGroup.main_id[j],'id_friend': dataFriends['response']['items'][jx]['id'], 'first_name': dataFriends['response']['items'][jx]['first_name'], 'last_name': dataFriends['response']['items'][jx]['last_name']}, ignore_index=True)
          GraphCentrality.add_edge(frameGroup.first_name[j] + " " + frameGroup.last_name[j], dataFriends['response']['items'][jx]['first_name'] + " " + dataFriends['response']['items'][jx]['last_name'])
      except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"


#2.1 проверка на связь друзей 2 уровня
for n in range(len(frameFriends.id_friend.unique())):
  try: ndataFriends = RequestsFriendsGet(frameFriends.id_friend.unique(), n)
  except: continue
  ncountOfUserFriends = ndataFriends['response']['count']
  for nx in range(ndataFriends['response']['count']): 
    try:
      if ndataFriends['response']['items'][nx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      if ndataFriends['response']['items'][nx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    except:  
      try: 
        if ndataFriends['response']['items'][nx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        else: 
          for ny in range(len(frameFriends.id_friend)):
            if ndataFriends['response']['items'][nx]['id'] == frameFriends.id_friend[ny]:
              GraphCentrality.add_edge(frameFriends.first_name[ny] + " " + frameFriends.last_name[ny], ndataFriends['response']['items'][nx]['first_name'] + " " + ndataFriends['response']['items'][nx]['last_name'])
              break
      except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"
  del ndataFriends

#3 уровень (получаем информацию о друзьях друзей одногруппников)
#for k in range(len(frameFriends.main_id)):
  #dataFriendsOfFrineds = RequestsFriendsGet(frameFriends.main_id, k)
  #countOfUserFriendsOfFriends = dataFriendsOfFrineds['response']['count']
  #for kx in range(countOfUserFriendsOfFriends):
    #try:
      #if dataFriendsOfFrineds['response']['items'][kx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      #if dataFriendsOfFrineds['response']['items'][kx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    #except:  
      #try: 
        #if dataFriendsOfFrineds['response']['items'][kx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        #else: 
          #frameFriendsOfFriends = frameFriendsOfFriends.append({'main_id': frameFriends.main_id[j],'id_friend': dataFriendsOfFrineds['response']['items'][jx]['id'], 'first_name': dataFriendsOfFrineds['response']['items'][jx]['first_name'], 'last_name': dataFriendsOfFrineds['response']['items'][jx]['last_name']}, ignore_index=True)
          #GraphCentrality.add_edge(frameFriends.first_name[k] + " " + frameFriends.last_name[k], dataFriends['response']['items'][kx]['first_name'] + " " + dataFriends['response']['items'][kx]['last_name'])
      #except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"


#3.1 проверка на связь друзей 3 уровня
#for m in range(len(frameFriendsOfFriends.id_friend.unique())):
  #mdataFriendOfFriends = RequestsFriendsGet(frameFriendsOfFriends.id_friend.unique(), m)
  #mcountOfUserFriends = mdataFriendOfFriends['response']['count']
  #for mx in range(mcountOfUserFriends): 
    #try:
      #if mdataFriendOfFriends['response']['items'][mx]['deactivated'] == "banned": banned = "Сработало исключение - banned"
      #if mdataFriendOfFriends['response']['items'][mx]['deactivated'] == "deleted": deleted = "Сработало исключение - deleted"
    #except:  
      #try: 
        #if mdataFriendOfFriends['response']['items'][mx]['is_closed'] == True: ClosedAcc = "Доступ к странице пользователя недоступен"
        #else: 
          #for my in range(len(frameFriendsOfFriends.id_friend)):
            #if mdataFriendOfFriends['response']['items'][mx]['id'] == frameFriendsOfFriends.id_friend[my]:
              #GraphCentrality.add_edge(frameFriendsOfFriends.first_name[my] + " " + frameFriendsOfFriends.last_name[my], mdataFriendOfFriends['response']['items'][mx]['first_name'] + " " + mdataFriendOfFriends['response']['items'][mx]['last_name'])
              #break
      #except: errorVK = "Ошибка сервера ВК - не получен доступ к аккаунту"

In [ ]:
pos = nx.spring_layout(GraphCentrality,scale=40, k=0.30, iterations=20)
nx.draw(GraphCentrality, pos, font_size=6, node_size=750, alpha=0.7, with_labels=True, edge_color='gray')
nx.draw(GraphCentrality)

In [ ]:
close_centrality = nx.closeness_centrality(GraphCentrality)
print("Оценка центральности по посредничеству")
close_centrality = sorted(close_centrality.items(), key=lambda item: item[1], reverse=True)
for i in range(10):
  print(close_centrality[i])

Оценка центральности по посредничеству
('Mikhail Nikiforov', 0.472399549380398)
('Dmitry Medvedev', 0.4166252690842855)
('Evgeny Malinovsky', 0.41239141124405837)
('Andrey Maslov', 0.3977236800505849)
('Artem Novikov', 0.3842394624312767)
('Igor Kotilevets', 0.38289453660021305)
('Dmitry Vlasov', 0.3761961722488038)
('Sergey Ivanov', 0.3733491615966761)
('Leonid Saburov', 0.36896905704648775)
('Anna Zaytseva', 0.36342626029178104)


In [ ]:
bet_centrality = nx.betweenness_centrality(GraphCentrality, normalized = True, endpoints = False)
print("Оценка центральности по близости")
bet_centrality = sorted(bet_centrality.items(), key=lambda item: item[1], reverse=True)
for i in range(10):
  print(bet_centrality[i])

Оценка центральности по близости
('Mikhail Nikiforov', 0.6669530994196431)
('Andrey Maslov', 0.25715274378919784)
('Evgeny Malinovsky', 0.1505000705920614)
('Nikita Skvortsov', 0.142347269087647)
('Egor Ruzaikin', 0.12437163528421231)
('Dmitry Medvedev', 0.10922676986579842)
('Ruslan Chkadua', 0.10795940442258023)
('Anna Zaytseva', 0.08962779140987624)
('Shokhin Gulyamov', 0.0825018094928047)
('Dmitry Vlasov', 0.06512090948497067)


In [ ]:
eigenvector_centrality = nx.eigenvector_centrality_numpy(GraphCentrality)
print("Оценка центральности по собственному вектору")
eigenvector_centrality = sorted(eigenvector_centrality.items(), key=lambda item: item[1], reverse=True)
for i in range(10):
  print(eigenvector_centrality[i])

Оценка центральности по собственному вектору
('Mikhail Nikiforov', 0.7060896549210794)
('Evgeny Malinovsky', 0.04369264372060153)
('Dmitry Medvedev', 0.034747599891845414)
('Leonid Saburov', 0.02384350381098965)
('Masha Chernysheva', 0.023766255377507893)
('Radion Korolyov', 0.023766255377507893)
('Danila Shavel', 0.02376625537750789)
('Denis Raev', 0.023766255377507886)
('Alexander Ermakov', 0.023766255377507883)
('Anastasia Konyushko', 0.023766255377507883)
